# Create a catalog of low-z galaxies

Details:
https://rubinobs.atlassian.net/browse/SP-1896

NED query:
```
RA: Unconstrained
DEC: Between -90d00m00.0000s and +40d00m00.0000s
GLON: Unconstrained
GLAT: Unconstrained
Redshift: Less Than 0.100000
Flux density: Unconstrained
Flux ratio: Unconstrained
Include ANY Object Type: Galaxies
Exclude Object Type:
Parameters for Distances and Cosmology: Ho= 73.0; Ωmatter = 0.27; Ωvacuum = 0.73;
Correct Redshift To the Reference Frame defined by the 3K CMB
Include objects with ANY of the 2 name prefixes selected: "NGC" , "IC"
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

## Create CSV file.

Read from file "ned_results.txt".

Write just the name, RA, Dec, and redshift to "lowz_galaxies.csv".

Print a few first lines to make sure you've skipped the header appropriately.

In [ ]:
fnm_in = 'ned_results.txt'
fnm_out = 'lowz_galaxies.csv'
fout = open(fnm_out, 'w')
fout.write('# name, ra, dec, redshift \n')

i = 0
for line in open(fnm_in, 'r'):
    if i >= 22:
        if i < 25:
            print(line)
        x = line.split('|')
        tmp1 = x[1].strip(' ')
        tmp2 = x[2].strip(' ')
        tmp3 = x[3].strip(' ')
        tmp4 = x[6].strip(' ')
        fout.write(tmp1 + ', ' + tmp2 + ', ' + tmp3 + ', ' + tmp4 + ' \n')
        del x, tmp1, tmp2, tmp3, tmp4
    i += 1
fout.close()

del i, fnm_in, fnm_out, fout

## Characterize the low-z galaxy set.

Read the data into numpy arrays.

In [ ]:
fnm = "lowz_galaxies.csv"
gal_name = np.loadtxt(fnm, dtype='str', usecols=(0), delimiter=',')
gal_ra = np.loadtxt(fnm, dtype='float', usecols=(1), delimiter=',')
gal_dec = np.loadtxt(fnm, dtype='float', usecols=(2), delimiter=',')
gal_z = np.loadtxt(fnm, dtype='float', usecols=(3), delimiter=',')

Number of unique catalogs (by object name).

Number of galaxies from each unique catalog).

In [ ]:
temp = []
for name in gal_name:
    tmp = name.split(' ')
    temp.append(tmp[0])
gal_cats = np.asarray(temp, dtype='str')
del temp

values, counts = np.unique(gal_cats, return_counts=True)
print('Number of unique catalogs by object name: ', len(values))

print(' ')
Nmax = 10
print('The top ', Nmax, ' catalogs: ')
sx = np.flip(np.argsort(counts))
for i, x in enumerate(sx):
    if i < Nmax:
        print('%-15s %5i' % (values[x], counts[x]))

Sky coverage.

In [ ]:
fig = plt.figure(figsize = (6, 2))
plt.plot(gal_ra, gal_dec, 'o', ms=3, alpha=0.3, mew=0, color='black')
plt.xlabel('RA')
plt.ylabel('Dec')
plt.show()

Redshift distribution.

In [ ]:
fig = plt.figure(figsize = (6, 2))
plt.hist(gal_z, bins=110, log=True)
plt.xlabel('redshift')
plt.show()

How many had negative velocities (negative redshifts)?

In [ ]:
tx = np.where(gal_z <= 0.0)[0]
print(len(tx), '(', np.round(100.0 * len(tx) / len(gal_z), 2) , '%)')

# for x in tx:
#     print(gal_name[x], gal_z[x])

How many galaxies might be duplicates?

In [ ]:
templist = ['NED01', 'NED02', 'NED03', 'NED04', 'NED05']
for temp in templist:
    tally = 0
    for name in gal_name:
        if name.find(temp) >= 0:
            tally += 1
    print(temp, tally)
del templist, tally

How many galaxies have nearby neighbors within 30"?

In [ ]:
radius = 30.0 #arcsec
gal_nn = np.zeros(len(gal_name), dtype='int')
for i, name in enumerate(gal_name):
    temp = np.sqrt(((np.cos(np.deg2rad(gal_dec[i]))) * (gal_ra[i] - gal_ra))**2 + 
                   (gal_dec[i] - gal_dec)**2)
    tx = np.where((temp > 0) & (temp < radius/3600.0))[0]
    gal_nn[i] = len(tx)
    del temp, tx

values, counts = np.unique(gal_nn, return_counts = True)
for value, count in zip(values, counts):
    print(value, count)